In [1]:
import numpy as np
import pandas as pd
import transformers
from transformers import AutoTokenizer
import datasets

import nltk
from nltk.corpus import brown # POS data

c:\Users\Aman\anaconda3\envs\huggingfacev3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
nltk.download('brown')
nltk.download('universal_tagset')

In [ ]:
def get_sentence_pos(sentences_pos_tuple):
    sentence = " ".join([word for word, pos in sentences_pos_tuple])
    pos = " ".join([pos for word, pos in sentences_pos_tuple])

    return (sentence, pos)



In [ ]:
corpus = brown.tagged_sents(tagset="universal")

In [ ]:
from tqdm import tqdm

In [ ]:
distinct_pos = []

In [ ]:
sentence_pos_dict = pd.DataFrame(columns=['sentence','pos'])
i=0
distinct_pos = []
for sentences_pos_tuple in tqdm(corpus, total=len(corpus)):
    sentence, pos = get_sentence_pos(sentences_pos_tuple)
    a = pos.split(" ")
    distinct_pos = distinct_pos + a
    distinct_pos = list(set(distinct_pos))
    sentence_pos_dict.loc[i,"sentence"]=sentence
    sentence_pos_dict.loc[i,"pos"]=pos
    i+=1
distinct_pos = list(set(distinct_pos))


In [ ]:
distinct_pos = list(set(distinct_pos))

In [ ]:
import pickle
from sklearn.model_selection import train_test_split
sentence_pos_dict.columns = ['sentence','label']
train,test = train_test_split(sentence_pos_dict)

train.to_csv("./data/train.csv",index=False)
test.to_csv("./data/test.csv",index=False)
with open('distinct_pos.pkl', 'wb') as file:
    pickle.dump(distinct_pos, file)

In [2]:
import pickle
with open('distinct_pos.pkl', 'rb') as file:
    # Step 3: Use pickle.load to deserialize the content
    distinct_pos = pickle.load(file)

In [3]:
# make id2label & label2id
id2label = {i:value for i, value in enumerate(distinct_pos)}
label2id = {value:key for key, value in id2label.items()}

In [4]:
# Tokenizer
checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


In [7]:
t = tokenizer(sentence_pos_dict.loc[0,"sentence"])
t

NameError: name 'sentence_pos_dict' is not defined

In [8]:
t.word_ids()

NameError: name 't' is not defined

In [5]:
from datasets import load_dataset
# Load the datasets from CSV files
dataset = load_dataset(
    "csv",
    data_files={'train': './data/train.csv', 'test': './data/test.csv'}
)


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 43005
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 14335
    })
})

In [7]:
tokenizer(dataset['train'][0]['sentence'], )

{'input_ids': [101, 1124, 1674, 1136, 1928, 1106, 1474, 1115, 3379, 1575, 1103, 27466, 3080, 12888, 10308, 1104, 1875, 1105, 1117, 25211, 1194, 1103, 2303, 132, 132, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:


# Tokenize for both input and target(label)
def tokenize_fn(batch):
    # Tokenize the input seq first
    # It will populate inputs_ids, attention_mask etc
    tokenized_inputs = tokenizer(batch['sentence'], truncation=True)

    # Tokenized_output using pos mapping
    label_batch_str = batch['label']
    label_batch_int = []
    for label_str in label_batch_str:
        pos_int=[-100]
        for pos in label_str.split(" "):
            #print(pos,label_str)
            pos_int.append(label2id[pos])
        pos_int+[-100]
        label_batch_int.append(pos_int)

    tokenized_inputs['labels'] = label_batch_int
    return tokenized_inputs

In [9]:
tokenized_datasets = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=dataset['train'].column_names # Removing column other than input_ids, attention_mask, labels
)

Map: 100%|██████████| 14335/14335 [00:00<00:00, 16061.53 examples/s]


In [10]:
tokenizer.decode(tokenized_datasets['train'][100]['input_ids'])

'[CLS] ) The sorry fact about this young man, who was barely of age when he broke into major - league baseball, was that he really was a better ball player than he was given credit for being - - never so good as he claimed, and always an irritant to his associates, but a good steady performer when he could fight down the temptation to orate on his skills or cut up in public. [SEP]'

In [11]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label = id2label,
    label2id = label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

train_args = TrainingArguments(
    "distilbert-finetuned-pos",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.1,
)

c:\Users\Aman\anaconda3\envs\huggingfacev3\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [20]:
from transformers import Trainer
trainer = Trainer(
    model = model,
    args = train_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    
    #compute_metrics=compute_metric,
    tokenizer=tokenizer
)

In [21]:
trainer.train()

  3%|▎         | 501/16128 [01:38<50:19,  5.18it/s]  

{'loss': 0.2288, 'grad_norm': 9.21403980255127, 'learning_rate': 1.937996031746032e-05, 'epoch': 0.09}


  6%|▌         | 1001/16128 [03:05<51:30,  4.89it/s] 

{'loss': 0.2232, 'grad_norm': 20.19095802307129, 'learning_rate': 1.8759920634920635e-05, 'epoch': 0.19}


  9%|▉         | 1501/16128 [04:43<44:19,  5.50it/s]  

...

                                                     
100%|██████████| 16128/16128 [55:58<00:00,  7.97it/s]

{'eval_loss': 0.10742051154375076, 'eval_runtime': 71.3245, 'eval_samples_per_second': 200.983, 'eval_steps_per_second': 25.125, 'epoch': 3.0}


100%|██████████| 16128/16128 [56:02<00:00,  4.80it/s]

{'train_runtime': 3362.9097, 'train_samples_per_second': 38.364, 'train_steps_per_second': 4.796, 'train_loss': 0.15166743028731572, 'epoch': 3.0}


TrainOutput(global_step=16128, training_loss=0.15166743028731572, metrics={'train_runtime': 3362.9097, 'train_samples_per_second': 38.364, 'train_steps_per_second': 4.796, 'total_flos': 1664451124297704.0, 'train_loss': 0.15166743028731572, 'epoch': 3.0})

In [22]:
trainer.save_model("my_saved_model")

In [26]:
trainer.push_to_hub("amanpatkar/distilbert-finetuned-pos", token = "<>")


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]



training_args.bin: 100%|██████████| 5.11k/5.11k [00:00<00:00, 11.9kB/s]










































































































































































































































































































































































































































































































































































































































































































































































































model.safetensors: 100%|██████████| 261M/261M [01:37<00:00, 2.67MB/s]
Upload 2 LFS files:

CommitInfo(commit_url='https://huggingface.co/amanpatkar/distilbert-finetuned-pos/commit/b64a65980fed80e1cf9c3ed44213c8315eeca958', commit_message='amanpatkar/distilbert-finetuned-pos', commit_description='', oid='b64a65980fed80e1cf9c3ed44213c8315eeca958', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
tokenizer.push_to_hub("amanpatkar/distilbert-finetuned-pos", token = "<>")

c:\Users\Aman\anaconda3\envs\huggingfacev3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Aman\.cache\huggingface\hub\models--amanpatkar--distilbert-finetuned-pos. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/amanpatkar/distilbert-finetuned-pos/commit/b67e5711d20288b02e5c1cfdeeb8ab00ccb20ac8', commit_message='Upload tokenizer', commit_description='', oid='b67e5711d20288b02e5c1cfdeeb8ab00ccb20ac8', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
from transformers import pipeline

In [28]:
pos = ner = pipeline(
    "token-classification",
    model = "amanpatkar/distilbert-finetuned-pos",
    aggregation_strategy = "simple",
    device = 0
)

In [29]:
s = "Aman Patkar owns the Honda KTM showroom in India. He is a boy."
pos(s)

[{'entity_group': 'NOUN',
  'score': np.float32(0.9963277),
  'word': 'Aman',
  'start': 0,
  'end': 4},
 {'entity_group': 'VERB',
  'score': np.float32(0.99982905),
  'word': 'Pat',
  'start': 5,
  'end': 8},
 {'entity_group': 'DET',
  'score': np.float32(0.99995303),
  'word': '##kar',
  'start': 8,
  'end': 11},
 {'entity_group': 'NOUN',
  'score': np.float32(0.99914724),
  'word': 'owns the Honda',
  'start': 12,
  'end': 26},
 {'entity_group': 'ADP',
  'score': np.float32(0.9999267),
  'word': 'K',
  'start': 27,
  'end': 28},
 {'entity_group': 'NOUN',
  'score': np.float32(0.99967134),
  'word': '##TM',
  'start': 28,
  'end': 30},
 {'entity_group': '.',
  'score': np.float32(0.99757427),
  'word': 'show',
  'start': 31,
  'end': 35},
 {'entity_group': 'PRON',
  'score': np.float32(0.930186),
  'word': '##room',
  'start': 35,
  'end': 39},
 {'entity_group': 'VERB',
  'score': np.float32(0.9140066),
  'word': 'in',
  'start': 40,
  'end': 42},
 {'entity_group': 'NOUN',
  'score':